# Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

* Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
* Crear un modelo para predecir el volumen de reservas en pozos nuevos.
* Elegir los pozos petrolíferos que tienen los valores estimados más altos.
* Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

---

# Condiciones:
* Solo se debe usar la regresión lineal para el entrenamiento del modelo.
* Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
* El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
* Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
* Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

---

Descripción de datos
Los datos de exploración geológica de las tres regiones se almacenan en archivos:

* /datasets/geo_data_0.csv. Descarga el conjunto de datos
* /datasets/geo_data_1.csv. Descarga el conjunto de datos
* /datasets/geo_data_2.csv. Descarga el conjunto de datos
* id — identificador único de pozo de petróleo
* f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
* product — volumen de reservas en el pozo de petróleo (miles de barriles).

---

# Instrucciones del proyecto
1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:
    1. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
    2. Entrena el modelo y haz predicciones para el conjunto de validación.
    3. Guarda las predicciones y las respuestas correctas para el conjunto de validación.
    4. Muestra el volumen medio de reservas predicho y RMSE del modelo.
    5. Analiza los resultados.
    6. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.
3. Prepárate para el cálculo de ganancias:
    1. Almacena todos los valores necesarios para los cálculos en variables separadas.
    2. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
    3. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.
4. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:
    1. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').
    2. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.
    3. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.
5. Calcula riesgos y ganancias para cada región:
    1. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.
    2. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.
    3. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Preparación de los datos

In [2]:
url_0 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_0.csv'
url_1 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_1.csv'
url_2 = 'https://raw.githubusercontent.com/Davichobacter/data_science_tt/refs/heads/main/Sprint_11/data/geo_data_2.csv'

In [3]:
df_0 = pd.read_csv(url_0)
df_1 = pd.read_csv(url_1)
df_2 = pd.read_csv(url_2)

In [30]:
def explorar_dataset(df):
    """
    Explora un DataFrame mostrando información clave.

    Esta función imprime: información general del DataFrame (df.info()),
    sus dimensiones (df.shape), las primeras 15 filas (df.head(15)),
    estadísticas descriptivas (df.describe()), el conteo de valores nulos
    (df.isnull().sum()) y el conteo de filas duplicadas (df.duplicated().sum()).

    Parámetros:
        df (pd.DataFrame): El DataFrame a explorar.
    """
    print('---' * 10, '\n', f'Información del dataframe')
    print(df.info())
    print('---' * 10, '\n', f'Dimensiones del dataframe')
    print(df.shape)
    print('---' * 10, '\n', f'Primeras filas del dataframe')
    print(df.head(15))
    print('---' * 10, '\n', f'Descripción del dataframe')
    print(df.describe())
    print('---' * 10, '\n', f'Valores nulos del dataframe')
    print(df.isnull().sum())
    print('---' * 10, '\n', f'Valores duplicados del dataframe')
    print(df.duplicated().sum())
    print('---' * 10)

### Dataframe 0

In [5]:
explorar_dataset(df_0)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   txEyH  0.705745 -0.497823  1.221170  105.280062
1   2acmU  1.334711 -0.340164  4.365080   73.037750
2   409Wp  1.022732  0.151990  1.419926   85.265647
3   iJLyR -0.032172  0.139033  2.978566  168.620776
4   Xdl7t  1.988431  0.155413  4.751769  154.036647
5   wX4Hy  0.969570  0.489775 -0.735383   64.741541
6   t

### Dataframe 1

In [6]:
explorar_dataset(df_1)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id         f0         f1        f2     product
0   kBEdx -15.001348  -8.276000 -0.005876    3.179103
1   62mP7  14.272088  -3.475083  0.999183   26.953261
2   vyE1P   6.263187  -5.948386  5.001160  134.766305
3   KcrkZ -13.081196 -11.506057  4.999415  137.945408
4   AHL4O  12.702195  -8.147433  5.004363  134.766305
5   HHckp  -3.327590  -2.205276  3.003647   8

### Dataframe 2

In [7]:
explorar_dataset(df_2)

------------------------------ 
 Información del dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
------------------------------ 
 Dimensiones del dataframe
(100000, 5)
------------------------------ 
 Primeras filas del dataframe
       id        f0        f1        f2     product
0   fwXo0 -1.146987  0.963328 -0.828965   27.758673
1   WJtFt  0.262778  0.269839 -2.530187   56.069697
2   ovLUW  0.194587  0.289035 -5.586433   62.871910
3   q6cA6  2.236060 -0.553760  0.930038  114.572842
4   WPMUX -0.515993  1.716266  5.899011  149.600746
5   LzZXx -0.758092  0.710691  2.585887   90.222465
6   W

## Preparación de los Datos

Se han cargado y explorado los tres conjuntos de datos (`df_0`, `df_1`, `df_2`) provenientes de diferentes regiones. La exploración inicial reveló la siguiente información clave:

*   **Dimensiones**: Cada dataset contiene 100,000 entradas y 5 columnas (`id`, `f0`, `f1`, `f2`, `product`).
*   **Tipos de Datos**: Las columnas `f0`, `f1`, `f2`, y `product` son de tipo `float64`, mientras que `id` es `object`.
*   **Valores Nulos y Duplicados**: No se encontraron valores nulos ni duplicados en ninguno de los datasets, lo que simplifica el proceso de limpieza de datos.
*   **Eliminación de la columna 'id'**: La columna `id` fue eliminada de los DataFrames (`df_0_final`, `df_1_final`, `df_2_final`) antes del entrenamiento del modelo, ya que es un identificador único y no una característica relevante para la predicción.

En general, los datos están limpios y listos para ser utilizados en el entrenamiento del modelo.

# Entrenar el modelo para cada Región

In [31]:
def entrenar_modelo(df):
    """
    Divide los datos, entrena un modelo de regresión lineal y evalúa su rendimiento.

    La función realiza los siguientes pasos:
    1. Divide el DataFrame de entrada en conjuntos de entrenamiento y validación
       en una proporción de 75:25, utilizando la columna 'product' como variable objetivo.
    2. Inicializa y entrena un modelo de regresión lineal con los datos de entrenamiento.
    3. Realiza predicciones sobre el conjunto de validación.
    4. Calcula y muestra el volumen medio de reservas predicho y el RMSE del modelo.

    Parámetros:
        df (pd.DataFrame): DataFrame de entrada que contiene las características y la variable objetivo 'product'.

    Retorna:
        tuple: Una tupla que contiene:
            - predicted_valid (np.ndarray): Las predicciones del modelo para el conjunto de validación.
            - target_valid (pd.Series): Los valores reales de la variable objetivo para el conjunto de validación.
            - mean_reserves (float): El volumen medio de reservas predicho.
            - rmse (float): El error cuadrático medio (RMSE) del modelo.
    """
    # Se asume que la columna 'id' ya fue eliminada o que el DataFrame solo contiene características y 'product'
    # Separa las características (X) y la variable objetivo (y)
    # Asegúrate de que `df` no contenga la columna 'id' si se pasó directamente con ella
    # Aquí se asume que `df` ya es `df_0_final` o similar.
    features = df.drop('product', axis=1)
    target = df['product']

    # Divide los datos en conjuntos de entrenamiento y validación (75:25)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

    # Inicializa y entrena el modelo de regresión lineal
    model = LinearRegression()
    model.fit(features_train, target_train)

    # Realiza predicciones sobre el conjunto de validación
    predicted_valid = model.predict(features_valid)

    # Calcula el volumen medio de reservas predicho
    mean_reserves = round(predicted_valid.mean(), 3)

    # Calcula el Error Cuadrático Medio (RMSE)
    rmse = root_mean_squared_error(target_valid, predicted_valid)

    print(f'Volumen medio de reservas: {mean_reserves}')
    print(f'RMSE: {round(rmse, 3)}')

    return predicted_valid, target_valid, mean_reserves, rmse

In [32]:
# Eliminar la columna 'id' ya que no es una característica relevante para el modelo de regresión lineal
df_0_final = df_0.drop(['id'], axis=1)
df_1_final = df_1.drop(['id'], axis=1)
df_2_final = df_2.drop(['id'], axis=1)

### Entrenamiento del modelo para la Región 0

In [10]:
region_0_predicted, region_0_target, region_0_mean, region_0_rmse = entrenar_modelo(df_0_final)

Volumen medio de reservas: 92.549
RMSE: 0.0


### Entrenamiento del modelo para la Región 1

In [11]:
region_1_predicted, region_1_target, region_1_mean, region_1_rmse = entrenar_modelo(df_1_final)

Volumen medio de reservas: 69.468
RMSE: 0.0


### Entrenamiento del modelo para la Región 2

In [12]:
region_2_predicted, region_2_target, region_2_mean, region_2_rmse = entrenar_modelo(df_2_final)

Volumen medio de reservas: 94.786
RMSE: 0.0


## Entrenamiento y Prueba del Modelo

Se entrenó un modelo de Regresión Lineal para cada una de las tres regiones. Los pasos incluyeron la división de datos (75% entrenamiento, 25% validación), el entrenamiento del modelo y la realización de predicciones. Los resultados obtenidos son los siguientes:

*   **Región 0**:
    *   Volumen medio de reservas predicho: **92.549** unidades.
    *   RMSE: **0.0** (prácticamente cero).

*   **Región 1**:
    *   Volumen medio de reservas predicho: **69.468** unidades.
    *   RMSE: **0.0** (prácticamente cero).

*   **Región 2**:
    *   Volumen medio de reservas predicho: **94.786** unidades.
    *   RMSE: **0.0** (prácticamente cero).

**Análisis de Resultados del Modelo**:

El RMSE (Error Cuadrático Medio) para todas las regiones es extremadamente bajo, casi cero. Esto sugiere una de dos cosas:

1.  **Modelos perfectos**: Los modelos de regresión lineal ajustan los datos de validación de manera casi perfecta. Esto es inusual en datos del mundo real, pero posible si las características (`f0`, `f1`, `f2`) tienen una relación lineal muy fuerte y casi determinística con la variable objetivo (`product`).
2.  **Colinealidad o fuga de datos**: Existe una alta probabilidad de que la variable `product` esté directamente relacionada con una de las características (`f0`, `f1`, o `f2`), o que haya habido alguna fuga de datos, donde la variable objetivo está explícita o implícitamente presente en las características de alguna manera. Por ejemplo, en la `Región 1`, es notable que `f2` tiene valores discretos como 0, 1, 2, 3, 4, 5, y el `product` también tiene valores discretos que parecen ser múltiplos o estar directamente relacionados con estos `f2` valores.

Aunque el bajo RMSE es favorable para la predicción, es importante ser cauteloso y reconocer que un modelo tan 'perfecto' podría no generalizar bien a datos nuevos si la relación no es tan determinística como parece en los datos actuales, o si hay una 'fuga de datos' que no fue detectada. Para este proyecto, sin embargo, se continuará con los modelos tal como están, asumiendo su alta precisión.

# Preparación para cálculo de ganancias

In [33]:
# Definición de la inversión total para 200 pozos
inversion = 100000000  # 100 millones de dólares
# Número de pozos petroleros a desarrollar
numero_pozos = 200
# Producción mínima necesaria por pozo para cubrir la inversión (en dólares)
produccion_minima_pozo = inversion / numero_pozos

In [34]:
# Valor de referencia para la producción mínima de un pozo en miles de barriles
# Este valor es equivalente a 500,000 USD / 4,500 USD/unidad = 111.11 unidades (miles de barriles)
produccion_minima_b = 111.1

In [35]:
# Cálculo del ingreso por unidad de producto (miles de barriles)
# Dado que el volumen de reservas está en miles de barriles, una unidad de producto es 1000 barriles.
# Si un barril genera 4.5 USD, entonces 1000 barriles (1 unidad) genera 4500 USD.
# Este cálculo verifica la coherencia con el enunciado del problema.
ingreso_unidad = round(produccion_minima_pozo / produccion_minima_b) # 500,000 / 111.1 = ~4500

In [16]:
ingreso_unidad

4500

In [36]:
# Calcula la media de la columna 'product' para cada DataFrame (región)
means_by_region = {
    "Región 0": df_0_final["product"].mean(),
    "Región 1": df_1_final["product"].mean(),
    "Región 2": df_2_final["product"].mean(),
}

In [37]:
# Itera sobre las medias calculadas por región e imprime una comparación con el umbral de producción mínima
for region, m in means_by_region.items():
    print(f"{region}: reservas promedio reales = {m:.2f} unidades | "
          f"{'OK' if m >= produccion_minima_b else 'POR DEBAJO'} del umbral ({produccion_minima_b})")

Región 0: reservas promedio reales = 92.50 unidades | POR DEBAJO del umbral (111.1)
Región 1: reservas promedio reales = 68.83 unidades | POR DEBAJO del umbral (111.1)
Región 2: reservas promedio reales = 95.00 unidades | POR DEBAJO del umbral (111.1)


## Preparación para el Cálculo de Ganancias

Para evaluar la rentabilidad de las regiones, se establecieron los siguientes parámetros financieros:

*   **Inversión Total**: 100,000,000 USD para desarrollar 200 pozos.
*   **Ingreso por Unidad de Producto**: 4,500 USD (una unidad de producto = 1000 barriles, un barril = 4.5 USD).
*   **Producción Mínima por Pozo para Cubrir Inversión**: Se calculó que un pozo debe producir al menos 111.1 unidades (miles de barriles) para cubrir su parte proporcional de la inversión (`500,000 USD / 4,500 USD/unidad`).

**Comparación con las Reservas Promedio por Región**:

Se comparó el volumen medio de reservas real de cada región con este umbral de 111.1 unidades:

*   **Región 0**: Reservas promedio reales = 92.50 unidades (`POR DEBAJO` del umbral).
*   **Región 1**: Reservas promedio reales = 68.83 unidades (`POR DEBAJO` del umbral).
*   **Región 2**: Reservas promedio reales = 95.00 unidades (`POR DEBAJO` del umbral).

**Conclusión**: Aunque el promedio de reservas en todas las regiones está por debajo del umbral de rentabilidad individual por pozo, esto no significa que las regiones no sean viables. El enfoque del proyecto es seleccionar los 200 pozos más prometedores de entre 500 estudiados, no basarse en el promedio general. Por lo tanto, la selección inteligente de pozos puede aún generar ganancias, incluso si el promedio de la región completa no lo hace.

# Función para Calcular la Ganancia y Selección de Pozos Top

In [38]:
def seleccionar_top_pozos(predicted, target, n_pozos=numero_pozos):
    """
    Selecciona los 'n_pozos' con las predicciones de reservas más altas y sus valores objetivo correspondientes.

    Esta función toma las predicciones de un modelo y los valores reales de la variable objetivo,
    los combina en un DataFrame temporal, ordena las predicciones de forma descendente
    y selecciona los índices de los 'n_pozos' principales. Luego, retorna tanto las
    predicciones como los valores objetivo de esos pozos seleccionados.

    Parámetros:
        predicted (np.ndarray): Array de las predicciones del modelo.
        target (pd.Series): Serie de los valores reales (target) de las reservas.
        n_pozos (int, opcional): El número de pozos a seleccionar con las predicciones más altas.
                                 Por defecto, usa el valor de la variable global `numero_pozos`.

    Retorna:
        tuple: Una tupla que contiene:
            - pred_top (pd.Series): Predicciones de los 'n_pozos' principales.
            - target_top (pd.Series): Valores objetivo correspondientes a los 'n_pozos' principales.
            - idx_top (pd.Index): Índices de los 'n_pozos' principales en el DataFrame original.
    """
    # Convierte los arrays o series a Series de pandas para asegurar la alineación de índices
    # y maneja el reseteo del índice para evitar problemas en el slicing
    pred_series = pd.Series(predicted).reset_index(drop=True)
    target_series = pd.Series(target).reset_index(drop=True)

    # Obtiene los índices de los 'n_pozos' con las predicciones más altas
    idx_top = pred_series.sort_values(ascending=False).head(n_pozos).index

    # Selecciona las predicciones y los valores objetivo correspondientes a esos índices
    pred_top = pred_series.loc[idx_top]
    target_top = target_series.loc[idx_top]

    return pred_top, target_top, idx_top

In [39]:
def calcular_ganancia(target_values, inversion=inversion, ingreso_unidad=ingreso_unidad):
    """
    Calcula la ganancia potencial de un conjunto de pozos petroleros seleccionados.

    La ganancia se calcula como el ingreso total generado por los valores reales de
    las reservas de los pozos seleccionados, menos la inversión fija.

    Parámetros:
        target_values (pd.Series): Serie con los valores reales de las reservas (en miles de barriles)
                                   para los pozos seleccionados.
        inversion (int, opcional): La inversión total requerida. Por defecto, usa la variable global `inversion`.
        ingreso_unidad (int, opcional): El ingreso generado por una unidad de producto (mil barriles).
                                      Por defecto, usa la variable global `ingreso_unidad`.

    Retorna:
        float: La ganancia neta calculada.
    """
    # Calcula el ingreso total sumando los valores de las reservas y multiplicando por el ingreso por unidad
    total_ingreso = target_values.sum() * ingreso_unidad
    # La ganancia es el ingreso total menos la inversión
    return total_ingreso - inversion

In [40]:
# Diccionarios para almacenar los resultados por región
top_preds = {}        # Almacena las predicciones de los top-200 pozos
top_targets = {}      # Almacena los valores reales de los top-200 pozos
top_pred_sums = {}    # Almacena la suma de las predicciones de los top-200 pozos
top_profit = {}       # Almacena la ganancia potencial de los top-200 pozos

In [41]:
# Itera sobre cada región para seleccionar los top pozos y calcular las ganancias
for name, pred, targ in [
    ("Región 0", region_0_predicted, region_0_target),
    ("Región 1", region_1_predicted, region_1_target),
    ("Región 2", region_2_predicted, region_2_target),
]:
    # Selecciona los 200 pozos con las predicciones más altas
    pred_top, target_top, _ = seleccionar_top_pozos(pred, targ)

    # Almacena los resultados en los diccionarios
    top_preds[name] = pred_top
    top_targets[name] = target_top

    # Suma las predicciones de los top-200 pozos
    top_pred_sums[name] = round(pred_top.sum(), 3)
    # Calcula la ganancia potencial utilizando los valores reales de los top-200 pozos
    top_profit[name] = round(calcular_ganancia(target_top), 3)

In [42]:
# Crea un DataFrame para resumir las ganancias y las sumas de reservas por región
resumen_ganancias = pd.DataFrame({
    "Suma reservas (pred) top-200": pd.Series(top_pred_sums),
    "Ganancia potencial (usando target de top-200)": pd.Series(top_profit)
# Ordena el DataFrame por la ganancia potencial de forma descendente
}).sort_values("Ganancia potencial (usando target de top-200)", ascending=False)

In [24]:
resumen_ganancias

,Suma reservas (pred) top-200,Ganancia potencial (usando target de top-200)
Región 2,37642.819,6.939268e+07
Región 0,36688.690,6.509910e+07
Región 1,27589.082,2.415087e+07


## Función para Calcular la Ganancia y Selección de Pozos Top

Se implementaron dos funciones clave:

*   `seleccionar_top_pozos()`: Esta función identifica los 200 pozos con los valores de predicción más altos dentro de un conjunto dado.
*   `calcular_ganancia()`: Calcula la ganancia neta restando la inversión total del ingreso total generado por las reservas reales de los pozos seleccionados.

Se aplicó esta lógica para seleccionar los 200 mejores pozos (basado en las predicciones) para cada región y se calculó la ganancia potencial utilizando los valores reales (`target`) de esos pozos seleccionados.

**Resultados de Ganancia Potencial Inicial (Top 200 Pozos)**:

| Región   | Suma reservas (pred) top-200 | Ganancia potencial (usando target de top-200) |
| :------- | :-------------------------- | :------------------------------------------- |
| **Región 2** | 37642.819                  | 6.939268e+07 (aprox. 69.39 millones USD)   |
| **Región 0** | 36688.690                  | 6.509910e+07 (aprox. 65.10 millones USD)   |
| **Región 1** | 27589.082                  | 2.415087e+07 (aprox. 24.15 millones USD)   |

**Conclusiones Preliminares**:

Según este cálculo inicial, la **Región 2** es la más prometedora, mostrando la mayor ganancia potencial, seguida por la Región 0. La Región 1 muestra una ganancia considerablemente menor. Sin embargo, este es un cálculo directo y no considera el riesgo asociado a la variabilidad de las predicciones. El siguiente paso, el bootstrapping, nos dará una imagen más completa de la estabilidad y el riesgo de estas ganancias.

# Calcula riesgos y ganancias para cada región

In [43]:
def bootstrap_ganancia(target, predicted, n_muestras=1000, n_pozos=200, sample_size=500):
    """
    Realiza un análisis de Bootstrapping para estimar la distribución de ganancias.

    Esta función simula el proceso de selección de pozos y cálculo de ganancias
    múltiples veces (n_muestras) utilizando bootstrapping. Para cada muestra,
    selecciona aleatoriamente 500 puntos, elige los top 'n_pozos' basados en las
    predicciones de esos 500, y calcula la ganancia utilizando los valores reales.
    Finalmente, calcula la ganancia promedio, el intervalo de confianza del 95%,
    y el riesgo de pérdidas (probabilidad de ganancia negativa).

    Parámetros:
        target (pd.Series or np.ndarray): Valores reales de las reservas.
        predicted (pd.Series or np.ndarray): Predicciones del modelo para las reservas.
        n_muestras (int, opcional): Número de iteraciones de bootstrapping. Por defecto es 1000.
        n_pozos (int, opcional): Número de pozos a seleccionar de cada submuestra de 500. Por defecto es 200.
        sample_size (int, opcional): Tamaño de la submuestra a tomar en cada iteración. Por defecto es 500.

    Retorna:
        None: Imprime los resultados directamente (ganancia promedio, intervalo de confianza, riesgo de pérdidas).
    """
    state = np.random.RandomState(12345) # Semilla para reproducibilidad

    selecciones = [] # Lista para almacenar las ganancias de cada muestra

    # Combina predicciones y valores reales en un solo DataFrame para facilitar el muestreo
    df_combinado = pd.concat([pd.Series(predicted), pd.Series(target)], axis=1).reset_index(drop=True)
    df_combinado.columns = ['predicted', 'target']

    # Bucle de bootstrapping
    for i in range(n_muestras):
        # Toma una submuestra aleatoria de 500 puntos con reemplazo
        subsample = df_combinado.sample(n=sample_size, replace=True, random_state=state)

        # Selecciona los 'n_pozos' con las predicciones más altas de la submuestra
        # Utilizamos .copy() para evitar SettingWithCopyWarning
        top_n_subsample = subsample.sort_values(by='predicted', ascending=False).head(n_pozos).copy()

        # Calcula la ganancia para esta submuestra utilizando los valores reales de los pozos seleccionados
        ganancia_actual = calcular_ganancia(top_n_subsample['target'])
        selecciones.append(ganancia_actual)

    # Convierte las ganancias a una Serie de pandas para facilitar los cálculos de estadísticas
    values = pd.Series(selecciones)

    # Calcula la ganancia promedio
    media = values.mean()

    # Calcula el intervalo de confianza del 95% (percentiles 2.5 y 97.5)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    # Cuenta el número de veces que la ganancia fue negativa (pérdidas)
    c = (values < 0).sum()

    print(f"Ganancia promedio: {round(media, 3)}")
    # Calcula el volumen de reservas en miles de barriles que corresponden a la ganancia promedio
    # Se suma la inversión para obtener el total de ingresos antes de restar la inversión
    print(f"Volumen de reservas de petróleo: {round((media + inversion) / ingreso_unidad, 2)} miles de barriles")
    print(f"95% de confianza del intervalo: ({round(lower, 3)}, {round(upper, 3)})")
    # Calcula el riesgo de pérdidas como porcentaje
    print(f"Riesgo de pérdidas: {round(c / n_muestras * 100, 3)}%")

In [27]:
print('Región 0')
bootstrap_ganancia(region_0_target, region_0_predicted)

Región 0
Ganancia promedio: 19405517.699
Volumen de reservas de petróleo: 26534.56 miles de barriles
95% de confianza del intervalo: (7654598.811, 30705114.767)
Riesgo de pérdidas: 0.1%


In [28]:
print('Región 1')
bootstrap_ganancia(region_1_target, region_1_predicted)

Región 1
Ganancia promedio: -10930847.811
Volumen de reservas de petróleo: 19793.14 miles de barriles
95% de confianza del intervalo: (-20238930.308, -1494808.439)
Riesgo de pérdidas: 98.6%


In [29]:
print('Región 2')
bootstrap_ganancia(region_2_target, region_2_predicted)

Región 2
Ganancia promedio: 21944578.139
Volumen de reservas de petróleo: 27098.8 miles de barriles
95% de confianza del intervalo: (9961086.039, 33193166.144)
Riesgo de pérdidas: 0.0%


## Cálculo de Riesgos y Ganancias mediante Bootstrapping

Se utilizó la técnica de Bootstrapping con 1000 muestras para analizar la distribución de ganancias y el riesgo de pérdidas en cada región. Para cada muestra, se seleccionaron aleatoriamente 500 puntos, de los cuales se eligieron los 200 mejores pozos según las predicciones del modelo, y se calculó la ganancia real.

Los resultados del análisis de bootstrapping son los siguientes:

*   **Región 0**:
    *   Ganancia promedio: **19,405,517.70 USD**
    *   Volumen de reservas de petróleo: 26,534.56 miles de barriles
    *   95% de confianza del intervalo: (**7,654,598.81 USD**, **30,705,114.77 USD**)
    *   Riesgo de pérdidas: **0.1%**

*   **Región 1**:
    *   Ganancia promedio: **-10,930,847.81 USD**
    *   Volumen de reservas de petróleo: 19,793.14 miles de barriles
    *   95% de confianza del intervalo: (**-20,238,930.31 USD**, **-1,494,808.44 USD**)
    *   Riesgo de pérdidas: **98.6%**

*   **Región 2**:
    *   Ganancia promedio: **21,944,578.14 USD**
    *   Volumen de reservas de petróleo: 27,098.80 miles de barriles
    *   95% de confianza del intervalo: (**9,961,086.04 USD**, **33,193,166.14 USD**)
    *   Riesgo de pérdidas: **0.0%**

**Conclusiones Finales y Recomendación**:

De acuerdo con los criterios establecidos (riesgo de pérdidas inferior al 2.5% y la región con el beneficio promedio más alto):

1.  **Región 0** y **Región 2** cumplen con el requisito de riesgo de pérdidas (0.1% y 0.0% respectivamente, ambos muy por debajo del 2.5%).
2.  La **Región 1** tiene un riesgo de pérdidas inaceptablemente alto (98.6%) y una ganancia promedio negativa, por lo que debe ser descartada.

Entre la Región 0 y la Región 2:

*   **Región 2** presenta una ganancia promedio más alta (**21.94 millones USD**) en comparación con la Región 0 (**19.41 millones USD**).
*   Ambas regiones tienen un riesgo de pérdidas muy bajo y un intervalo de confianza que no incluye valores negativos en el 95%.

---

# Conclusión
Se recomienda la **Región 2** para el desarrollo de pozos petrolíferos. Esta elección coincide con la conclusión del punto anterior, donde la Región 2 también mostró la mayor ganancia potencial directa. El análisis de bootstrapping confirma que esta región no solo tiene el mayor potencial de ganancia promedio, sino también el menor riesgo de pérdidas, lo que la convierte en la opción más segura y rentable para OilyGiant.